# Road Rage: Finding the Ideal Speed Limit

### Assumptions
* Drivers want to go up to 120 km/hr.
* The average car is 5 meters long.
* Drivers want at least a number of meters equal to their speed in meters/second between them and the next car.
* Drivers will accelerate 2 m/s<sup>2</sup> up to their desired speed as long as they have room to do so.
* If another car is too close, drivers will match that car's speed until they have room again.
* If a driver would hit another car by continuing, they stop.
* Drivers will randomly (10% chance each second) slow by 2 m/s.
* This section of road is one lane going one way.
* Assume that drivers enter the road at the speed they left.
* Simulation starts with 30 cars per kilometer, evenly spaced.

## Normal Mode
We have a 1 kilometer section of road being built and do not know what the speed limit should be. This notebook simulates the 1 kilometer of road. Even though this road is not circular, the simulation treats it as such in order to generate a continuous flow of traffic.

In [1]:
import math
import matplotlib.pyplot as plt
import numpy as np
import random
from traffic_lib import *
%matplotlib inline

In [44]:
class Car:
    """
    Responsibilities:
    - Know speed (in m/s)
    - Know distance to driver ahead
    - Keep distance from driver ahead
    - Accelerate if possible
    - Match speed of those ahead if within safety zone
    - Stop if new location would result in crash (0 distance to car ahead)
    """
    
    def __init__(self, location, gap=28, speed_limit=33, start_speed=28):
        self.location = location
        self.gap = gap
        self.desired_speed = speed_limit
        self.speed = start_speed
        self.size = 5
        self.bumper = 0
        self.update_bumper()

    def __str__(self):
        return 'Car(location={},gap={},speed={})'.format(self.location, self.gap, self.speed)
    
    def __repr__(self):
        return self.__str__()
    
    def drive(self, car_ahead):

        if self.stop(car_ahead):
            pass
            
        elif self.match_speed(car_ahead):
            pass
            
        elif self.random_slowdown(car_ahead):
            pass
            
        elif self.accelerate(car_ahead):
            pass
            
        self.update_location(car_ahead)

    def stop(self, car_ahead):
        if self.location + self.speed >= car_ahead.bumper:
            self.speed = 0
            self.location = car_ahead.bumper - 1
            return True
        
    def match_speed(self, car_ahead):
        if self.location + self.speed < car_ahead.bumper - self.speed:
            self.speed = car_ahead.speed
            return True
        
    def random_slowdown(self, car_ahead):
        if random.random() < 0.1:
            self.speed -= 2
            if self.speed < 0:
                self.speed = 0
            return True
    
    def accelerate(self, car_ahead):
        if self.speed < self.desired_speed:
            if self.desired_speed - self.speed <= 2:
                self.speed = self.desired_speed
            else:
                self.speed += 2
            return True
    
    def update_location(self, car_ahead):
        if self.location + self.speed < 1000:
            self.location = self.location + self.speed
        else:
            self.location = self.location + self.speed - 1000
        self.update_bumper()
        self.gap = car_ahead.bumper - self.location * self.speed
    
    def update_bumper(self):
        if self.location >= self.size - 1:
            self.bumper = self.location - (self.size - 1)
        else:
            self.bumper = self.location - self.size + 1000
            # Note that to make the bumper location correct, we need to add 1 when adding 1000

In [45]:
class Road:
    """
    Responsibilities:
    - Hold length of road
    - Keep a list of vehicles on road
        - Initialize with number of cars
        - (1000 - sum(vehicle.size)) // len(self.vehicles)
    - Hold potential for slowdown
    
    Collaborators:
    - Car
    """
    def __init__(self, num_trucks=0):
        self.total_vehicle_space = ((30 - num_trucks) * 5) + (num_trucks * 25)
        self.initial_gap = (1000 - self.total_vehicle_space) // 30
        self.vehicles = [Car((4 + 33*n), self.initial_gap) for n in range(30 - num_trucks)]
        
        if num_trucks > 0:
            [self.vehicles.append(Truck() for _ in range(num_trucks))]

        self.vehicles[-1].gap = (1000 - self.vehicles[-1].location)
        
    def reinsert_car(self):
        if self.vehicles[-1].location < self.vehicles[-1].desired_speed + 1:
            last_car = self.vehicles.pop(-1)
            self.vehicles.insert(0, Car(last_car.location, last_car.gap))
            self.vehicles[0].update_bumper()
            return True

In [46]:
class HighwaySim:
    """
    Responsibilities:
    - Have road place cars at beginning of road when they reach the end
    - Keep track of time (seconds)
    - Step through time (ticks)
        - For each car on Road, tell car behind new situation and allow it to react
    - Report stats; return traffic jam status
    - Pop, Append
    
    Collaborators:
    - Car
    - Road
    """
    
    def __init__(self):
        self.road = Road()
        self.ticks = 0
        self.is_traffic = []
    
    def run_sim(self, duration=1):
        while self.ticks < duration:
            self.iterate()
            self.ticks += 1
        if self.is_traffic.count(True) > 0:
            return True
        else:
            return False
    
    def iterate(self):
        for idx, v in enumerate(self.road.vehicles):
            if idx > 0:
                car_ahead = self.road.vehicles[- idx - 1]
            else:
                car_ahead = Car(self.road.vehicles[0].location + 1000, self.road.vehicles[0].gap, 33,
                                self.road.vehicles[0].speed)
            
            v.drive(car_ahead)
            
            if v.speed == 0:
                self.is_traffic.append(True)
            else:
                self.is_traffic.append(False)
        
        for n in range(30):
            self.road.reinsert_car()

In [47]:
sim1 = HighwaySim()
sim1.run_sim()
sim1.road.vehicles

[Car(location=27,gap=28,speed=28),
 Car(location=32,gap=104,speed=28),
 Car(location=65,gap=-896,speed=28),
 Car(location=98,gap=-1853,speed=28),
 Car(location=131,gap=-2810,speed=28),
 Car(location=164,gap=-3767,speed=28),
 Car(location=197,gap=-4724,speed=28),
 Car(location=230,gap=-5681,speed=28),
 Car(location=263,gap=-6638,speed=28),
 Car(location=296,gap=-7595,speed=28),
 Car(location=329,gap=-8552,speed=28),
 Car(location=362,gap=-9509,speed=28),
 Car(location=395,gap=-10466,speed=28),
 Car(location=428,gap=-11423,speed=28),
 Car(location=461,gap=-12380,speed=28),
 Car(location=496,gap=-14385,speed=30),
 Car(location=491,gap=492,speed=0),
 Car(location=456,gap=457,speed=0),
 Car(location=423,gap=424,speed=0),
 Car(location=390,gap=391,speed=0),
 Car(location=357,gap=358,speed=0),
 Car(location=324,gap=325,speed=0),
 Car(location=291,gap=292,speed=0),
 Car(location=258,gap=259,speed=0),
 Car(location=225,gap=226,speed=0),
 Car(location=192,gap=193,speed=0),
 Car(location=159,gap=

In [40]:
def highway_trials(num_trials=1000):
    """
    Run num_trials of run_sim
    """
    trial_stats = []
    for _ in range(num_trials):
        trial_data = HighwaySim()
        trial_stats.append(trial_data)
    return trial_stats

In [41]:
highway_1000_data = highway_trials()

In [11]:
print(highway_1000_data.count(True))

0
